In [1]:
%load_ext autoreload
%autoreload 2
%pylab inline
pylab.rcParams['figure.figsize'] = (12, 8)
import warnings; warnings.filterwarnings("ignore")

import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import sys; sys.path.append('../')
from PDE_FIND import *
import scipy.io as sio
import itertools

import numpy as np
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D
from sympy import symbols

import sys; sys.path.append('../')
from PDE_FIND import *
from solvel0 import solvel0

import sys; sys.path.append('../../parametric-discovery/')
from best_subset import *
from frols import frols
from p_linear_regression import PLinearRegression
from r_pca import R_pca
from pde_diff_new import RPCA

import scipy.io as sio
from scipy.signal import savgol_filter
import itertools

from sklearn.preprocessing import normalize
from sklearn.linear_model import Ridge, BayesianRidge, LinearRegression as SkLinearRegression
from sklearn.base import BaseEstimator
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error
from abess.linear import LinearRegression
import pysindy as ps

from tqdm import trange
from os.path import join as join_path
FIGURE_EXPORT_PATH = "/Users/pongpisit/Documents/figure_export/"

from function_collections import msemse

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib
Sklearn's version: 1.2.2
mrmr is not installed in the env you are using. This may cause an error in future if you try to use the (missing) lib.


In [2]:
X_pre = np.load("./IPI_output_files/X_pre_noise1e-1_denoised.npy")
y_pre = np.load("./IPI_output_files/y_pre_noise1e-1_denoised.npy")
u_pre, v_pre = y_pre[:, 0:1], y_pre[:, 1:2]
del y_pre
best_subsets_u = np.load('./IPI_output_files/u/best_subsets_denoised.npy', allow_pickle=True)
best_subsets_v = np.load('./IPI_output_files/v/best_subsets_denoised.npy', allow_pickle=True)

In [3]:
from UBIC import *

In [4]:
tau = 3
thresholds = np.arange(0.001, 0.1+0.001, 0.001)
verbose = False
success_count = 0
all_count = 0

post_means, b_bics, b_uns = baye_uncertainties(best_subsets_u, (X_pre, u_pre), u_type='cv1', take_sqrt=True)
predictions = X_pre@post_means
lower_bounds = []
for k, efi in enumerate(best_subsets_u):
    com = len(efi)
    assert com == np.count_nonzero(post_means[:, k:k+1])
    lower_bound = 2*np.abs(log_like_value(predictions[:, k:k+1], u_pre))/np.log(len(u_pre))-com
    lower_bounds.append(lower_bound)
    
for thres in thresholds:
    last_lam = np.log10(max(lower_bounds/b_uns))
    delta = last_lam/tau
    now_lam = last_lam-delta
    last_ubic = UBIC(b_bics, b_uns, len(u_pre), hyp=10**last_lam)
    last_bc = np.argmin(last_ubic)
    while now_lam >= 0:
        now_ubic = UBIC(b_bics, b_uns, len(u_pre), hyp=10**now_lam)
        now_bc = np.argmin(now_ubic)

        diff_com = now_bc-last_bc
        diff_bic = b_bics[now_bc]-b_bics[last_bc]
        imp = abs(diff_bic/(b_bics[last_bc]*diff_com))
        if verbose:
            print(min(last_bc, now_bc), '<--->', max(last_bc, now_bc), np.nan_to_num(imp, nan=np.inf))

        if (diff_com > 0 and (diff_bic > 0 or imp < thres)) or \
            (diff_com < 0 and diff_bic > 0 and imp > thres):
            break

        last_lam = now_lam
        now_lam = round(last_lam-delta, 10)
        last_ubic = now_ubic
        last_bc = now_bc

    last_lam = round(last_lam, 10)
    last_lam_d_u = last_lam
    last_ubic_d_u = last_ubic
    last_bc_d_u = last_bc
    uns_u = b_uns
    
    if last_bc_d_u == 0:
        break
    elif last_bc_d_u == 5:
        success_count += 1
    all_count += 1
    print(last_bc_d_u, end=', ')
    
print("\nSuccess rate:", 100*success_count/all_count)

5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 
Success rate: 100.0


In [5]:
tau = 3
thresholds = np.arange(0.001, 0.1+0.001, 0.001)
verbose = False
success_count = 0
all_count = 0

post_means, b_bics, b_uns = baye_uncertainties(best_subsets_v, (X_pre, v_pre), u_type='cv1', take_sqrt=True)
predictions = X_pre@post_means
lower_bounds = []
for k, efi in enumerate(best_subsets_v):
    com = len(efi)
    assert com == np.count_nonzero(post_means[:, k:k+1])
    lower_bound = 2*np.abs(log_like_value(predictions[:, k:k+1], v_pre))/np.log(len(v_pre))-com
    lower_bounds.append(lower_bound)

for thres in thresholds:
    last_lam = np.log10(max(lower_bounds/b_uns))
    delta = last_lam/tau
    now_lam = last_lam-delta
    last_ubic = UBIC(b_bics, b_uns, len(v_pre), hyp=10**last_lam)
    last_bc = np.argmin(last_ubic)
    while now_lam >= 0:
        now_ubic = UBIC(b_bics, b_uns, len(v_pre), hyp=10**now_lam)
        now_bc = np.argmin(now_ubic)

        diff_com = now_bc-last_bc
        diff_bic = b_bics[now_bc]-b_bics[last_bc]
        imp = abs(diff_bic/(b_bics[last_bc]*diff_com))
        if verbose:
            print(min(last_bc, now_bc), '<--->', max(last_bc, now_bc), np.nan_to_num(imp, nan=np.inf))

        if (diff_com > 0 and (diff_bic > 0 or imp < thres)) or \
            (diff_com < 0 and diff_bic > 0 and imp > thres):
            break

        last_lam = now_lam
        now_lam = round(last_lam-delta, 10)
        last_ubic = now_ubic
        last_bc = now_bc

    last_lam = round(last_lam, 10)
    last_lam_d_v = last_lam
    last_ubic_d_v = last_ubic
    last_bc_d_v = last_bc
    uns_v = b_uns

    if last_bc_d_v == 0:
        break
    elif last_bc_d_v == 4:
        success_count += 1
    all_count += 1
    print(last_bc_d_v, end=', ')
    
print("\nSuccess rate:", 100*success_count/all_count)

4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 
Success rate: 100.0
